In [6]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, average_precision_score, precision_recall_curve
import matplotlib.pyplot as plt

In [10]:
try:
    df = pd.read_csv('../../../Downloads/IBD_estructured_text.csv', sep=',')
    print("¡Éxito! El archivo se leyó correctamente con el separador de coma.")
    display(df.head())
    df['created_utc'] = pd.to_datetime(df['created_utc'])
except Exception as e:
    print(f"Falló con la coma. Error: {e}")

¡Éxito! El archivo se leyó correctamente con el separador de coma.


,id,subreddit,author,title,selftext,created_utc,comments,cuerpo
0,dflwn,CrohnsDisease,zakool21,Don't be afraid of diagnostic procedures....,"I'm not likely to frontpage this subreddit, bu...",2010-09-18 11:59:11,"['I have to agree, the worse part of a Colonos...",TÍTULO:\nDon't be afraid of diagnostic procedu...
1,dfyy1,CrohnsDisease,sphinctersayzwha,Mayo Clinic article on Crohn's Disease.,Mayo Clinic article on Crohn's Disease.,2010-09-19 15:11:21,[],TÍTULO:\nMayo Clinic article on Crohn's Diseas...
2,dfz1m,CrohnsDisease,WeDeserveDessert,Has anyone else here taken Remicade? What did...,Has anyone else here taken Remicade? What did...,2010-09-19 15:19:54,"[""Remicade (and a bypass) changed my life. Wh...",TÍTULO:\nHas anyone else here taken Remicade? ...
3,dh6zd,CrohnsDisease,RosenTurd,"I was Diagnosed With Crohns Disease at age 9 ,...",Like the title Says AMA,2010-09-22 04:50:57,"[""My son was diagnosed at ten. It's hell for h...",TÍTULO:\nI was Diagnosed With Crohns Disease a...
4,dhgot,CrohnsDisease,unknownpleasures,Has anyone here had to have bowel surgery more...,I had a small bowel resection and appendectomy...,2010-09-22 18:31:36,"[""I have a story:\n\nI had a small bowel resec...",TÍTULO:\nHas anyone here had to have bowel sur...


In [11]:
assert 'label' in df.columns, "❌ El dataset necesita una columna 'label' con 0/1"

AssertionError: ❌ El dataset necesita una columna 'label' con 0/1

In [3]:
try:
    df = pd.read_csv(file)

    # Paso 4: Mostrar los primeros 10 registros del DataFrame
    # El método .head(n) nos devuelve las primeras 'n' filas del DataFrame.
    # Es una forma rápida de inspeccionar los datos y entender su estructura.
    print(f"Mostrando los primeros 10 registros del archivo: {file}")
    display(df.head(10))

except FileNotFoundError:
    print(f"Error: El archivo '{file}' no fue encontrado.")
    print("Por favor, asegúrate de haber subido el archivo y de que el nombre sea correcto.")


Mostrando los primeros 10 registros del archivo: ../../../Downloads/IBD_estructured_text.csv


,id,subreddit,author,title,selftext,created_utc,comments,cuerpo
0,dflwn,CrohnsDisease,zakool21,Don't be afraid of diagnostic procedures....,"I'm not likely to frontpage this subreddit, bu...",2010-09-18 11:59:11,"['I have to agree, the worse part of a Colonos...",TÍTULO:\nDon't be afraid of diagnostic procedu...
1,dfyy1,CrohnsDisease,sphinctersayzwha,Mayo Clinic article on Crohn's Disease.,Mayo Clinic article on Crohn's Disease.,2010-09-19 15:11:21,[],TÍTULO:\nMayo Clinic article on Crohn's Diseas...
2,dfz1m,CrohnsDisease,WeDeserveDessert,Has anyone else here taken Remicade? What did...,Has anyone else here taken Remicade? What did...,2010-09-19 15:19:54,"[""Remicade (and a bypass) changed my life. Wh...",TÍTULO:\nHas anyone else here taken Remicade? ...
3,dh6zd,CrohnsDisease,RosenTurd,"I was Diagnosed With Crohns Disease at age 9 ,...",Like the title Says AMA,2010-09-22 04:50:57,"[""My son was diagnosed at ten. It's hell for h...",TÍTULO:\nI was Diagnosed With Crohns Disease a...
4,dhgot,CrohnsDisease,unknownpleasures,Has anyone here had to have bowel surgery more...,I had a small bowel resection and appendectomy...,2010-09-22 18:31:36,"[""I have a story:\n\nI had a small bowel resec...",TÍTULO:\nHas anyone here had to have bowel sur...
5,diwa3,CrohnsDisease,SirFrancisDashwood,Itchy Butt,"Hi, \n\nIt's been about 8 years since I was d...",2010-09-25 23:20:25,['What about using a bidet to just relax that ...,"TÍTULO:\nItchy Butt\n\nCONTENIDO:\nHi, \n\nIt..."
6,dkeom,CrohnsDisease,cnstrctvcyncism,Seeking a psychologist/psychiatrist for Crohn's?,I feel like I do a good job dealing with the p...,2010-09-29 08:22:31,"[""I haven't tried it, but group therapy is a p...",TÍTULO:\nSeeking a psychologist/psychiatrist f...
7,dlt0i,CrohnsDisease,rxvette,Think Crohn's disease is a lifetime of drugs a...,Think Crohn's disease is a lifetime of drugs a...,2010-10-02 05:15:49,"[""Has anyone on Reddit tried Helminthic therap...",TÍTULO:\nThink Crohn's disease is a lifetime o...
8,dpm24,CrohnsDisease,[deleted],Skin problems related to Crohn's?,I was recently diagnosed with Crohn's. Almost ...,2010-10-11 07:23:48,"[""Sounds like erythema nodosum It's not rare w...",TÍTULO:\nSkin problems related to Crohn's?\n\n...
9,dsru9,CrohnsDisease,Antares42,Miracle Mineral Solution doesn't cure your Cro...,Miracle Mineral Solution doesn't cure your Cro...,2010-10-18 13:00:28,[],TÍTULO:\nMiracle Mineral Solution doesn't cure...
